In [31]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

In [32]:
WINDOW_SIZE = 25
NUMBER_OF_FEATURES = 4
NUMBER_OF_CLASSES = 2

MOTION_CLASSES = ['STAND', 'STEPS']
MOTIONTYPE_CLASSES = ['SML', 'MED', 'LAR']
MOTIONSPEED_CLASSES = ['SLOW', 'AVERAGE', 'FAST']


# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)

DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\TEST2\\'

# numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'

# v0 data (not normalized, use to generate more data)
ALL_X_TEST_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-CLASSIFY-V0.npy'
ALL_Y_TEST_CLASSIFY_MOTION_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CLASSIFY-MOTION-V0.npy'
ALL_Y_TEST_CLASSIFY_MOTIONTYPE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CLASSIFY-MOTIONTYPE-V0.npy'
ALL_Y_TEST_CLASSIFY_MOTIONSPEED_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CLASSIFY-MOTIONSPEED-V0.npy'

In [33]:
LABEL_TO_CATEGORY_MOTION = {label: category for category, label in enumerate(MOTION_CLASSES)}
LABEL_TO_CATEGORY_MOTIONTYPE = {label: category for category, label in enumerate(MOTIONTYPE_CLASSES)}
LABEL_TO_CATEGORY_MOTIONSPEED = {label: category for category, label in enumerate(MOTIONSPEED_CLASSES)}

print(LABEL_TO_CATEGORY_MOTION)
print(LABEL_TO_CATEGORY_MOTIONTYPE)
print(LABEL_TO_CATEGORY_MOTIONSPEED)

{'STAND': 0, 'STEPS': 1}
{'SML': 0, 'MED': 1, 'LAR': 2}
{'SLOW': 0, 'AVERAGE': 1, 'FAST': 2}


In [34]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and os.path.isfile(os.path.join(DATA_FOLDER, file))]

print(file_names)

['PROC-TEST2-STEPS-LR-LAR-130BPM.xlsx', 'PROC-TEST2-STEPS-LR-LAR-180BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-260BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-45BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-65BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-90BPM.xlsx', 'PROC-TEST2-STEPS-LR-MED-100BPM.xlsx', 'PROC-TEST2-STEPS-LR-MED-130BPM.xlsx', 'PROC-TEST2-STEPS-LR-MED-180BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-MED-200BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-MED-260BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-MED-45BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-MED-50BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-MED-65BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-MED-90BPM.xlsx']


In [35]:
def normalizeSensorData(sensor_input):
    return (sensor_input + 45)/90

In [36]:
def df2Xy(df, windowSize=5):
  X = []
  y_Motion= []
  y_MotionType = []
  y_MotionSpeed = []

  # NORMALIZE THE SENSOR DATA
  df["L_Pitch_Delta"] = df["L_Pitch_Delta"].apply(normalizeSensorData)
  df["L_Roll_Delta"] = df["L_Roll_Delta"].apply(normalizeSensorData)
  df["R_Pitch_Delta"] = df["R_Pitch_Delta"].apply(normalizeSensorData)
  df["R_Roll_Delta"] = df["R_Roll_Delta"].apply(normalizeSensorData)

  for i in range( len(df) - windowSize + 1):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values = df.loc[i + j, ['L_Pitch_Delta', 'L_Roll_Delta', 'R_Pitch_Delta', 'R_Roll_Delta']].values.tolist()

        # add row values to the input
        input.append(row_values)

    # add our input to our total inputs, marked as X
    X.append(input)

  y_Motion = df.iloc[windowSize - 1:, df.columns.get_loc('Class_Motion')].tolist()
  y_MotionType = df.iloc[windowSize - 1:, df.columns.get_loc('Class_MotionType')].tolist()
  y_MotionSpeed = df.iloc[windowSize - 1:, df.columns.get_loc('Class_MotionSpeed')].tolist()


  return (np.array(X), np.array(y_Motion), np.array(y_MotionType), np.array(y_MotionSpeed))

In [37]:
ALL_X = np.empty((0, WINDOW_SIZE, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_MOTION = np.empty((0))  # List to store all Y training data
ALL_Y_MOTIONTYPE = np.empty((0))  # List to store all Y training data
ALL_Y_MOTIONSPEED = np.empty((0))  # List to store all Y training data

In [38]:
def processData(ALL_X, ALL_Y_MOTION, ALL_Y_MOTIONTYPE, ALL_Y_MOTIONSPEED):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        xTest, yTest_Motion, yTest_MotionType, yTest_MotionSpeed = df2Xy(df, WINDOW_SIZE)
        
        ALL_X = np.concatenate([ALL_X, xTest])
        ALL_Y_MOTION = np.concatenate([ALL_Y_MOTION, yTest_Motion])
        ALL_Y_MOTIONTYPE = np.concatenate([ALL_Y_MOTIONTYPE, yTest_MotionType])
        ALL_Y_MOTIONSPEED = np.concatenate([ALL_Y_MOTIONSPEED, yTest_MotionSpeed])
        
        print('xTest.shape:', xTest.shape)
        print('ALL_X.shape:', ALL_X.shape, ' ALL_Y_MOTION.shape:', ALL_Y_MOTION.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # convert to numerical labels (originally text labels)
    numerical_motion_label = np.vectorize(LABEL_TO_CATEGORY_MOTION.get)(ALL_Y_MOTION)
    numerical_motiontype_label = np.vectorize(LABEL_TO_CATEGORY_MOTIONTYPE.get)(ALL_Y_MOTIONTYPE)
    numerical_motionspeed_label = np.vectorize(LABEL_TO_CATEGORY_MOTIONSPEED.get)(ALL_Y_MOTIONSPEED)
    

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TEST_CLASSIFY_PATH, ALL_X)
    np.save(ALL_Y_TEST_CLASSIFY_MOTION_PATH, numerical_motion_label)
    np.save(ALL_Y_TEST_CLASSIFY_MOTIONTYPE_PATH, numerical_motiontype_label)
    np.save(ALL_Y_TEST_CLASSIFY_MOTIONSPEED_PATH, numerical_motionspeed_label)


In [39]:
if not os.path.exists(ALL_X_TEST_CLASSIFY_PATH):
    processData(ALL_X, ALL_Y_MOTION, ALL_Y_MOTIONTYPE, ALL_Y_MOTIONSPEED)

ALL_X_TEST_CLASSIFY = np.load(ALL_X_TEST_CLASSIFY_PATH)
ALL_Y_TEST_CLASSIFY_MOTION = np.load(ALL_Y_TEST_CLASSIFY_MOTION_PATH)
ALL_Y_TEST_CLASSIFY_MOTIONTYPE = np.load(ALL_Y_TEST_CLASSIFY_MOTIONTYPE_PATH)
ALL_Y_TEST_CLASSIFY_MOTIONSPEED = np.load(ALL_Y_TEST_CLASSIFY_MOTIONSPEED_PATH)

print(ALL_X_TEST_CLASSIFY.shape)
print(ALL_Y_TEST_CLASSIFY_MOTION.shape)
print(ALL_Y_TEST_CLASSIFY_MOTIONTYPE.shape)
print(ALL_Y_TEST_CLASSIFY_MOTIONSPEED.shape)



xTest.shape: (3771, 25, 4)
ALL_X.shape: (3771, 25, 4)  ALL_Y_MOTION.shape: (3771,)
-------------------------------
xTest.shape: (3582, 25, 4)
ALL_X.shape: (7353, 25, 4)  ALL_Y_MOTION.shape: (7353,)
-------------------------------
xTest.shape: (3771, 25, 4)
ALL_X.shape: (11124, 25, 4)  ALL_Y_MOTION.shape: (11124,)
-------------------------------
xTest.shape: (7183, 25, 4)
ALL_X.shape: (18307, 25, 4)  ALL_Y_MOTION.shape: (18307,)
-------------------------------
xTest.shape: (7561, 25, 4)
ALL_X.shape: (25868, 25, 4)  ALL_Y_MOTION.shape: (25868,)
-------------------------------
xTest.shape: (3582, 25, 4)
ALL_X.shape: (29450, 25, 4)  ALL_Y_MOTION.shape: (29450,)
-------------------------------
xTest.shape: (3501, 25, 4)
ALL_X.shape: (32951, 25, 4)  ALL_Y_MOTION.shape: (32951,)
-------------------------------
xTest.shape: (3618, 25, 4)
ALL_X.shape: (36569, 25, 4)  ALL_Y_MOTION.shape: (36569,)
-------------------------------
xTest.shape: (3488, 25, 4)
ALL_X.shape: (40057, 25, 4)  ALL_Y_MOTION